In [674]:
%matplotlib notebook
from src.main.mando_preprocessing import *
import numpy as np
import matplotlib.pyplot as plt
import os

In [675]:
# Path to the raw data file

In [676]:
def load_data(path):
    # Find the weight file in the directory
    files = [f for f in os.listdir(path) if f.startswith('weights_') and f.endswith('.txt')]
    if not files:
        raise ValueError(f"No weight files found in directory: {path}")
    # We assume there's only one file matching the pattern, hence we take the first one
    file_path = os.path.join(path, files[0])
    # Load the data using pandas
    weights = pd.read_csv(file_path, header=None, names=['weight'])
    return weights['weight']

# The rest of the script remains the same, you just need to call this function in the main function


In [677]:
def moving_average(data, window_size):
    """Calculate the moving average of the given data using a window of specified size."""
    cumsum_vec = np.cumsum(np.insert(data, 0, 0)) 
    return (cumsum_vec[window_size:] - cumsum_vec[:-window_size]) / window_size

In [678]:
def create_decreasing_curve(data):
    """Ensure that the data only decreases or stays constant over time to represent only food consumption."""
    decreased_data = np.copy(data)
    for i in range(1, len(decreased_data)):
        if decreased_data[i] > decreased_data[i - 1]:
            decreased_data[i] = decreased_data[i - 1]
    return decreased_data

In [679]:
def plot(raw_data, decreasing_data):
    """
    Plot the raw, smoothed, and decreasing step-like weight data in separate subplots sharing the x-axis.

    Parameters:
    - raw_data (np.ndarray): The raw weight data array.
    - smoothed_data (np.ndarray): The smoothed weight data array.
    - decreasing_data (np.ndarray): The decreasing step-like weight data array.
    """
    fig, axs = plt.subplots(2, 1, figsize=(15, 10), sharex=True)
    
    # Raw data plot
    axs[0].plot(raw_data, label='Raw Data', alpha=0.5, color='green')
    axs[0].set_title('Raw Weight Data')
    axs[0].set_ylabel('Weight (grams)')
    axs[0].legend()

    # # Smoothed data plot
    # axs[1].plot(smoothed_data, label='Smoothed Data', color='orange')
    # axs[1].set_title('Smoothed Weight Data')
    # axs[1].set_ylabel('Weight (grams)')
    # axs[1].legend()

    # Decreasing step-like data plot
    axs[1].plot(decreasing_data, label='Decreasing Step-like Data', color='red')
    axs[1].set_title('Decreasing Step-like Weight Data')
    axs[1].set_xlabel('Time (arbitrary units)')
    axs[1].set_ylabel('Weight (grams)')
    axs[1].legend()
    
    plt.tight_layout()  # Adjust layout to prevent overlap
    plt.show()


In [680]:
def calculate_bite_sizes(decreasing_data, threshold=4):
    """
    Calculate the number of bites and the weight of each bite from the decreasing step-like weight data,
    considering only those bites where the weight decrease is greater than a specified threshold.

    Parameters:
    - decreasing_data (np.ndarray): The decreasing step-like weight data array.
    - threshold (int): The minimum weight decrease to qualify as a bite.

    Returns:
    - int: The number of bites.
    - list: The list of weights for each qualified bite.
    """
    bite_sizes = []
    for i in range(1, len(decreasing_data)):
        if decreasing_data[i] < decreasing_data[i - 1]:
            bite_size = decreasing_data[i - 1] - decreasing_data[i]
            if bite_size >= threshold:
                bite_sizes.append(bite_size)
    
    number_of_bites = len(bite_sizes)
    return number_of_bites, bite_sizes

# You would then call this function as before in the main function.


In [681]:
from scipy.signal import find_peaks, savgol_filter

def calculate_bite_sizes_advanced(decreasing_data, threshold=4, window_length=21, polyorder=3):
    """
    Advanced calculation of the number of bites and the weight of each bite using signal processing
    techniques to detect the local minima in the smoothed signal.

    Parameters:
    - decreasing_data (np.ndarray): The decreasing step-like weight data array.
    - threshold (int): The minimum weight decrease to qualify as a bite.
    - window_length (int): The length of the filter window (number of coefficients). Must be a positive odd integer.
    - polyorder (int): The order of the polynomial used to fit the samples. Must be less than `window_length`.

    Returns:
    - int: The number of bites.
    - list: The list of weights for each qualified bite.
    """
    # Apply a Savitzky-Golay filter to the data to smooth it while preserving peaks
    smoothed_data = savgol_filter(decreasing_data, window_length=window_length, polyorder=polyorder, mode='interp')

    # Compute the first derivative of the smoothed data
    derivative = np.diff(smoothed_data, n=1)
    
    # Find peaks (local minima) in the negative derivative (which correspond to bites in the original data)
    peaks, _ = find_peaks(-derivative, height=-threshold)

    # Calculate bite sizes based on the peaks detected
    bite_sizes = np.diff(peaks, prepend=0)
    
    # Filter out the consecutive bites that are too close to each other, if necessary
    # This step is optional and can be customized based on domain knowledge
    
    number_of_bites = len(bite_sizes)
    return number_of_bites, bite_sizes.tolist()

# You would call this function in your main function after processing the data.


In [682]:

path = '../data/raw/17'

# Load the raw data
weights = load_data(path).to_numpy()
window_size = 5
# smoothed_weights = moving_average(weights, window_size)
decreasing_data = create_decreasing_curve(weights)
trimmed_raw_weights = weights[window_size - 1:]
number_of_bites, bite_sizes = calculate_bite_sizes(decreasing_data,4)

print(f"Number of bites simple: {number_of_bites}")
print(f"Weight of each bite simple: {bite_sizes}")
# plot(trimmed_raw_weights, decreasing_data

adv_number_of_bites, adv_bite_sizes = calculate_bite_sizes_advanced(decreasing_data, 2, 7, 5)

print(f"Number of bites advanced: {adv_number_of_bites}")
print(f"Weight of each bite advanced: {adv_bite_sizes}")



Number of bites simple: 49
Weight of each bite simple: [27, 4, 9, 4, 20, 6, 13, 4, 11, 4, 8, 13, 13, 10, 12, 16, 13, 7, 15, 14, 6, 9, 18, 12, 8, 4, 8, 13, 11, 23, 6, 14, 11, 13, 20, 5, 7, 14, 7, 4, 5, 5, 15, 10, 7, 5, 11, 4, 4]
Number of bites advanced: 130
Weight of each bite advanced: [8, 3, 3, 15, 3, 3, 3, 7, 2, 3, 3, 9, 3, 3, 2, 3, 3, 3, 2, 3, 3, 7, 3, 3, 4, 3, 6, 3, 3, 5, 3, 5, 3, 4, 3, 4, 3, 3, 4, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 2, 3, 3, 7, 4, 3, 12, 3, 4, 5, 3, 2, 3, 5, 4, 3, 3, 3, 3, 4, 3, 5, 3, 3, 3, 3, 8, 3, 3, 3, 3, 3, 4, 3, 11, 3, 3, 3, 7, 3, 3, 4, 3, 3, 3, 3, 3, 4, 3, 3, 6, 3, 2, 3, 3, 3, 14, 3, 3, 4, 5, 3, 10, 3, 3, 3, 3, 13, 3, 3, 7, 3, 4, 2, 2, 4]
